In [1]:
#obtaining training data
classes = ["campus","graduate","SBU","undergraduate"]
data = []
label = []
train_data = []
train_label = []
test_data = []
test_label = []
class_data = []

for i,clas in enumerate(classes):
    with open("/Users/krishnasharma/Downloads/SBU_QA/doc_retrieval/topics/"+clas+"/sbu.txt","r") as f:
        paras = f.read()
        parasl = paras.split('\n')
        parasl = [para for para in parasl if len(para) > 20] 
        data += parasl
        class_data.append(parasl)
        label+= [i]*len(parasl)
        train_data += data[-len(parasl)//2:]
        train_label += label[-len(parasl)//2:]
        test_data += data[-len(parasl):-len(parasl)//2]
        test_label += label[-len(parasl):-len(parasl)//2]

In [2]:
#divide in test and training data

import numpy as np
twenty_train,twenty_test = {},{}
twenty_train["data"] = train_data
twenty_train["target"] = train_label  
twenty_test["data"] = train_data
twenty_test["target"] = train_label  



In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#svm classifier
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(twenty_train["data"], twenty_train["target"])
predicted_svm = text_clf_svm.predict(twenty_test["data"])
np.mean(predicted_svm == twenty_test["target"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


1.0

In [4]:
#top 5 nearest neighbor to find paragraph
def find_first_5_paragraph(q,c):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import NearestNeighbors

    import nltk
    def tokenize_and_stem(text):
        tokens = nltk.word_tokenize(text)
        stemmer = nltk.stem.porter.PorterStemmer()
        return [i for i in [stemmer.stem(t) for t in tokens] if len(i) > 2]

    tfidf_v = TfidfVectorizer(max_df=0.5, min_df=0, max_features=200000,
                   stop_words='english', use_idf=True, tokenizer=tokenize_and_stem)
    tfidf_weights_paras = tfidf_v.fit_transform([q]+class_data[c])

    # Create the k-NN model using k=5
    nn_paras = NearestNeighbors(n_neighbors=5, algorithm='auto')

    # Fit the models to the TF-IDF weights matrix
    nn_fitted_paras = nn_paras.fit(tfidf_weights_paras)
    dist,idx = nn_fitted_paras.kneighbors(tfidf_weights_paras[0,:])
    print(idx,len(class_data[c]))
    return [class_data[c][x-1] for x in idx[0]]

In [5]:
#ask me questions like - 
test_data = '''Which insurance is available for domestic stuents?
Who administrates SHIP?
What is the annual deductible amount for SHIP?
What is the Annual out of pocket limit?
Where is the fee billed?
Who should we contact for questions related to claims?
Who should we contact for questions related to insurance charges?
What is the cost of insurance for Fall 2018?
Who was second president of stony brook university?
When was stony brook university established?
Where is stony brook university located?
By what name stony brook athletic team is known?
How many employees does university have?
Who was governor of New york in 1960?
How many students were enrolled in 1962?
Who was forth president of stony brook univeristy?
What makes stony brook univeristy best values univeristy?
Which stadium was opened in 2002?
What is span of new computer science buiding?
Which part of the campus is center for academic life in Stony Brook University?
What was original name of university's athletic team?
To whom Joe Nathan Field is dedicated to?
On which LIRR line stony brook university located?
What are the application requirements for graduate school?,
How many departments does sbu have?
What insurance is available for domestic students?'''
test_data = test_data.split("\n")
predicted = text_clf_svm.predict(test_data)
print([classes[int(p)] for p in predicted])

['SBU', 'graduate', 'SBU', 'SBU', 'SBU', 'undergraduate', 'undergraduate', 'SBU', 'SBU', 'SBU', 'SBU', 'SBU', 'SBU', 'SBU', 'campus', 'SBU', 'SBU', 'campus', 'SBU', 'SBU', 'SBU', 'SBU', 'SBU', 'graduate', 'graduate', 'undergraduate']


In [11]:
paras = []
ques = []
for i,pred in enumerate(predicted):
    paras += find_first_5_paragraph(test_data[i],pred)
    ques += [test_data[i]]*5
print(len(paras),len(ques))

[[ 0 19 26 23 13]] 26
[[ 0 30 13 16 26]] 31
[[ 0 17 22  7 14]] 26
[[ 0  3 17 22  7]] 26
[[ 0 17 21 16 19]] 26
[[ 0 18 27 32 19]] 35
[[ 0 15  2  4 18]] 35
[[ 0 26 18  6 12]] 26
[[ 0 15 26  4 20]] 26
[[ 0  2 20 12 15]] 26
[[ 0 19 26 23 13]] 26
[[ 0 24 25  1  8]] 26
[[ 0 23  1 20 19]] 26
[[ 0  2 20  1 15]] 26
[[ 0 12 13  6  7]] 15
[[ 0  4 15  3  6]] 26
[[ 0  5  6 22 19]] 26
[[ 0 15  5  2  4]] 15
[[ 0  7 20  6 19]] 26
[[ 0  7  5 23 10]] 26
[[ 0 25 24  1  2]] 26
[[ 0  8 24  6 21]] 26
[[ 0 19 26 23 13]] 26
[[ 0  5 11  6  3]] 31
[[ 0 29 11  5 13]] 31
[[ 0 22 27  7 32]] 35
130 130


In [7]:
import re
import json
def create_json(contexts,ques):
    print("creating json file ....")
    para_dic = {}
    context_count = 0
    para_dic["paragraphs"] = []
    spam_filter = re.compile(r'\[\d.*\]')
    for i,context in enumerate(contexts): 
        context = spam_filter.sub('',context)
        if len(context) <= 50:
            continue   
        para_dic["paragraphs"].append({})
        para_dic["paragraphs"][context_count]["context"] = context  
        qas_count = 0
        para_dic["paragraphs"][context_count]["qas"] = []
        for q in [ques[i]]:
            para_dic["paragraphs"][context_count]["qas"].append({})
            para_dic["paragraphs"][context_count]["qas"][qas_count]["answers"] = []
            para_dic["paragraphs"][context_count]["qas"][qas_count]["answers"].append({})
            para_dic["paragraphs"][context_count]["qas"][qas_count]["answers"][0]["answer_start"] = 0
            para_dic["paragraphs"][context_count]["qas"][qas_count]["answers"][0]["text"] = ""
            para_dic["paragraphs"][context_count]["qas"][qas_count]["question"] = q
            para_dic["paragraphs"][context_count]["qas"][qas_count]["id"] = str(context_count)+str(qas_count)
            qas_count += 1
        context_count+= 1
    data = {"data":[para_dic]}
    with open("qna_sbu_test.json","w") as f:
        f.write(json.dumps(data))
    return json.dumps(data)

In [10]:
create_json(paras,ques)

creating json file ....


'{"data": [{"paragraphs": [{"context": "Stony Brook fans rush the field after a memorable comeback over the Colgate Raiders for a seventh straight Homecoming victory.However, the Seawolves returned to national prominence in 2011-12 season after an off year with six programs participating in NCAA sanctioned tournaments. The fall season took the football team to new heights with a record-breaking nine-game winning streak which led the team to a 9-4, 6-0 Big South season earning their third consecutive conference championship and first outright followed by an NCAA tournament berth which saw the Seawolves advance to the second round for the first time ever. The season included multiple sellout games at LaValle Stadium, a first for the program. Men\'s soccer returned to the NCAA tournament for the second time in three years after winning their third America East championship at home but fell in their first round match after penalty shootouts. The success continued with Women\'s volleyball a

In [9]:
#You can also generate json for you own context and paragraphs
custom_para = ['''Domestic Student Health Insurance Plan (SHIP) .Benefits and Highlights of the SHIP.SHIP 
has been developed especially for Stony Brook students (and their dependents) to provide access to comprehensive
care that complements the quality health services on campus.The details of the plan are reviewed and recommended
each year by committee members to ensure that the coverage is well-suited to the needs of the Stony Brook students 
and respectful of their budgets. SHIP is administered by United Healthcare. The Plans meet all of the student health
insurance standards developed by the American College Health Association.SHIP is tailor-made for the college
population.Provides continuous coverage at a reasonable cost for most on or off-campus medical care over 
Fall/Winter and Spring/Summer Semesters.Covers pre-existing medical conditions & preventative care.
Annual deductible $200 for an individual.Annual out of pocket limit of $3,000 which includes deductibles, 
copays and coinsurance.Covers inpatient and outpatient mental health care.No deductible applied to prescription 
drug coverage.Please note: Office visits for Primary Care and Specialists have a $35 copayment 
with 0% coinsurance with a referral and 30% coinsurance without a referral.''']
custom_ques = ["What is the annual deductible amount for SHIP?"]
json_file = create_json(custom_para,custom_ques)
json_file

creating json file ....


'{"data": [{"paragraphs": [{"context": "Domestic Student Health Insurance Plan (SHIP) .Benefits and Highlights of the SHIP.SHIP \\nhas been developed especially for Stony Brook students (and their dependents) to provide access to comprehensive\\ncare that complements the quality health services on campus.The details of the plan are reviewed and recommended\\neach year by committee members to ensure that the coverage is well-suited to the needs of the Stony Brook students \\nand respectful of their budgets. SHIP is administered by United Healthcare. The Plans meet all of the student health\\ninsurance standards developed by the American College Health Association.SHIP is tailor-made for the college\\npopulation.Provides continuous coverage at a reasonable cost for most on or off-campus medical care over \\nFall/Winter and Spring/Summer Semesters.Covers pre-existing medical conditions & preventative care.\\nAnnual deductible $200 for an individual.Annual out of pocket limit of $3,000 whi